## Library

In [1]:
import os 
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math
import operator
from math import sqrt

from collections import Counter
# Lemmatize with POS Tag
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn import linear_model


## Loading Data 

In [2]:
D = np.load('count_vector_matrix.npy') # load

In [3]:
D.shape

(9025, 19931)

In [4]:
smd = pd.read_csv('../the-movies-dataset/movies_metadata_equal_ratings.csv')
ratings = pd.read_csv('../the-movies-dataset/ratings_equal_movies_metadata.csv')
train = pd.read_csv('../the-movies-dataset/ratings_train.csv')
test = pd.read_csv('../the-movies-dataset/ratings_test.csv')

In [5]:
smd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9025 entries, 0 to 9024
Data columns (total 26 columns):
adult                    9025 non-null bool
belongs_to_collection    1670 non-null object
budget                   9025 non-null int64
genres                   9025 non-null object
homepage                 1954 non-null object
id                       9025 non-null int64
imdb_id                  9025 non-null object
original_language        9025 non-null object
original_title           9025 non-null object
overview                 9013 non-null object
popularity               9025 non-null float64
poster_path              9022 non-null object
production_companies     9025 non-null object
production_countries     9025 non-null object
release_date             9025 non-null object
revenue                  9025 non-null float64
runtime                  9025 non-null float64
spoken_languages         9025 non-null object
status                   9023 non-null object
tagline             

In [6]:
N = len(smd)
N

9025

In [7]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99810 entries, 0 to 99809
Data columns (total 6 columns):
userId        99810 non-null int64
id            99810 non-null float64
rating        99810 non-null float64
movieId       99810 non-null int64
timestamp     99810 non-null int64
movieIndex    99810 non-null int64
dtypes: float64(2), int64(4)
memory usage: 4.6 MB


In [8]:
ratings

,userId,id,rating,movieId,timestamp,movieIndex
0,0,152.0,2.5,1371,1260759135,1107
1,0,9909.0,2.5,31,1260759144,30
2,0,847.0,2.0,2193,1260759198,1739
3,0,9426.0,2.5,2455,1260759113,1958
4,0,6114.0,3.5,1339,1260759125,1079
...,...,...,...,...,...,...
99805,670,786.0,2.0,3897,1063503718,3108
99806,670,7443.0,4.0,3751,1065111939,2995
99807,670,1891.0,5.0,1196,1064890635,949
99808,670,279.0,4.0,1225,1065149143,977


In [9]:
train.shape

(79848, 6)

In [10]:
test.shape

(19962, 6)

In [11]:
n_users = len(ratings['userId'].unique().astype('int'))
n_items = len(ratings['id'].unique().astype('int'))
n_users

671

In [12]:
len(train[train['userId'] == 1])

63

In [13]:
len(test[test['userId'] == 1])

12

In [14]:
ratings_train = train[['userId','id','rating','movieIndex']].values
ratings_test = test[['userId','id','rating','movieIndex']].values

In [15]:
ratings_train.shape

(79848, 4)

In [16]:
# this fuction return movie_index in meta_data_dataset and ratings of user  
def get_items_rated_by_user(rate_matrix, user_id):
    y = rate_matrix[:,0] # get all the value in user_id column
    # get list of user_id index in y which value equal input 
    # user_id
    ids = np.where(y == user_id)[0]
    # get list movie_id matching user_id 
    item_ids = rate_matrix[ids, 3]
    item_ids = [int(i) for i in item_ids]
    
    scores = rate_matrix[ids,2]
    return (item_ids, scores)

In [17]:
a, b = get_items_rated_by_user(ratings_train, 0)
for i in a :
    print(i)

1083
1043
1661
1013
1739
856
1511
1811
30
903
1958
1079
830
1037


In [18]:
train[train['userId'] == 0]

,userId,id,rating,movieId,timestamp,movieIndex
0,0,1598.0,2.0,1343,1260759131,1083
1,0,783.0,2.0,1293,1260759148,1043
2,0,97.0,4.0,2105,1260759139,1661
3,0,11778.0,2.0,1263,1260759151,1013
4,0,847.0,2.0,2193,1260759198,1739
5,0,819.0,3.0,1061,1260759182,856
6,0,1051.0,4.0,1953,1260759191,1511
7,0,8916.0,2.0,2294,1260759108,1811
8,0,9909.0,2.5,31,1260759144,30
9,0,1103.0,2.0,1129,1260759185,903


In [19]:
D.shape[1]

19931

In [20]:
d = D.shape[1] # movie vector dimention
W = np.zeros((d, n_users)) 
b = np.zeros((1,n_users))

In [21]:
for i in range(n_users):
        idx, scores = get_items_rated_by_user(ratings_train, i)
        clf = clf = Ridge(alpha=1.0, random_state=241)
        Xhat = D[idx,:]
        clf.fit(Xhat, scores)
        W[:, i] = clf.coef_
        b[0, i] = clf.intercept_

In [22]:
Yhat = D.dot(W) + b 
Yhat.shape

(9025, 671)

In [23]:
def get_list_movie_have_not_rated():
    user_rated, score = get_items_rated_by_user(rate_matrix,userId)
    result = []
    for i in range(movie_count):
         if i not in user_rated: result.append(i)
    return (userId, result)            

In [24]:
def get_pred_rating(rate_matrix, userId,movie_count):
    userId, list_not_rate =  get_list_user_have_not_rated(rate_matrix, userId,movie_count)
    result = {}
    for i in list_not_rate:
        result[i] = Yhat[i, userId]
    
    result = sorted(result.items(), key=operator.itemgetter(1),reverse=True)
    list_movie = []
    for i in result:
        list_movie.append(i[0])
    return list_movie

In [25]:
#user_rated, score = get_items_rated_by_user(ratings_train,0)
#pd.DataFrame(smd.iloc[user_rated])

In [26]:
result = get_pred_rating(ratings_train,0,N)
pd.DataFrame(smd.iloc[result[0:10]])

NameError: name 'get_list_user_have_not_rated' is not defined

In [ ]:
n = 0
np.set_printoptions(precision=2)
ids, scores = get_items_rated_by_user(ratings_test, n)
result = Yhat[ids, n]
result

In [ ]:
#test[test['userId'] == 0]

In [ ]:
print("ratings prediction of user ",n," :")
for i in range(len(result)):
    print("movie id : ",ids[i]," - ","predict ratings : ",result[i]," - ","true ratings : ",scores[i])

In [ ]:
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred 
        se += (e*e).sum(axis = 0)
        cnt += e.size 
    return sqrt(se/cnt)

In [ ]:
print('RMSE for test case:', evaluate(Yhat, ratings_train, W, b))
print('RMSE for test case:', evaluate(Yhat, ratings_test, W, b))

In [ ]:
np.save('count_pred_rating.npy', Yhat)